In [1]:
# --- 필요한 패키지 설치 ---
#!pip install requests langchain python-dotenv notion-client

# --- 필요한 import ---
from datetime import datetime
import os
import requests
from langchain.docstore.document import Document
from notion_client import Client
from typing import List, Dict
from langchain.schema import Document
from dotenv import load_dotenv


############################################
# --- Slack (더미) ---
############################################

# --- .env 로드 ---
load_dotenv()
# --- 환경변수에서 읽기 ---
SLACK_TOKEN = os.getenv("SLACK_TOKEN", "")
SLACK_CHANNEL_ID = os.getenv("SLACK_CHANNEL_ID", "")


# --- 주간보고서 관련 키워드 (30개) ---
REPORT_KEYWORDS = [
    "회의", "미팅", "일정", "계획", "진행", "완료", "성과", "목표", "보고",
    "결과", "피드백", "공유", "자료", "문서", "슬라이드", "첨부", "업로드",
    "검토", "확인", "협의", "토론", "논의", "배포", "개발", "테스트",
    "이슈", "문제", "해결", "개선", "요청"
]

############################################
# --- 더미 메시지 생성 함수 (총 50개) ---
############################################
def generate_dummy_messages():
    """
    50개의 더미 Slack 메시지를 생성해서 반환
    (업무 관련 40개 + 잡담 10개)
    """
    messages = []
    base_ts = int(datetime(2025, 9, 15, 9, 0).timestamp())  # 2025-09-15 09:00 시작
    chat_channels = ["개발팀 채널", "QA팀 채널", "운영팀 채널", "기획팀 채널", "경영지원팀", "프로젝트A 그룹", "프로젝트B 그룹"]

    # 업무 관련 메시지 40개
    work_texts = [
        "오늘 회의는 오후 3시에 시작합니다.",
        "회의 자료는 구글 드라이브에 올려놨어요.",
        "프로젝트 일정표 업데이트했습니다.",
        "고객사 피드백을 공유드립니다.",
        "내일 오전에 코드 리뷰 진행하겠습니다.",
        "배포는 오후 6시에 시작할 예정입니다.",
        "신규 기능 개발 일정이 확정되었습니다.",
        "이번 주 목표는 성능 개선입니다.",
        "테스트 환경에서 문제가 발견되었습니다.",
        "이슈 해결 방안을 논의해야 합니다.",
        "다음 주 미팅 안건을 정리했습니다.",
        "고객 요청사항을 반영한 문서를 첨부합니다.",
        "이번 릴리즈 결과를 보고드립니다.",
        "팀원별 업무 진행 상황을 공유합니다.",
        "문제 해결 후 재배포가 필요합니다.",
        "성과를 정리한 슬라이드를 업로드했습니다.",
        "QA팀에서 테스트 결과 보고서를 공유했습니다.",
        "신규 프로젝트 계획안을 업로드했습니다.",
        "고객사 미팅 피드백을 반영해야 합니다.",
        "서버 장애 문제를 해결 완료했습니다.",
        "팀 목표 달성을 위해 개선 작업이 필요합니다.",
        "자료 검토 후 의견을 부탁드립니다.",
        "이번 주 요청사항을 정리했습니다.",
        "배포 완료 후 최종 결과를 보고합니다.",
        "성과 발표 슬라이드를 회의 전에 공유하겠습니다.",
        "테스트 진행 상황을 보고드립니다.",
        "신규 기능 개발 완료 보고서를 첨부합니다.",
        "회의 피드백을 반영한 수정안을 공유합니다.",
        "이번 주 일정이 변경되었습니다.",
        "고객 요청에 따른 추가 개발이 필요합니다.",
        "프로젝트 리스크 검토 회의를 잡겠습니다.",
        "성과 발표 준비 자료를 업로드했습니다.",
        "이번 주 배포 계획을 정리했습니다.",
        "회의록을 기반으로 개선 사항을 정리했습니다.",
        "테스트 중 발견된 문제를 보고합니다.",
        "문서 검토 후 서명을 부탁드립니다.",
        "성과 달성을 위한 팀 목표를 공유합니다.",
        "이번 주 완료된 작업 리스트를 보고합니다.",
        "회의에서 나온 요청사항을 정리했습니다.",
        "신규 프로젝트 킥오프 미팅 일정을 공유합니다."
    ]

    # 잡담 메시지 10개
    casual_texts = [
        "오늘 점심 뭐 먹을래?",
        "야식 같이 시켜요.",
        "주말에 영화 보러 갈 사람?",
        "ㅋㅋㅋㅋㅋ",
        "오늘 날씨 진짜 좋다.",
        "헬스장 사람 너무 많네.",
        "고양이 사진 올립니다 🐱",
        "이번에 산 게임 재밌다!",
        "택배 언제 오지?",
        "출근하기 싫다..."
    ]

    # --- 업무 메시지 40개 생성 ---
    for i in range(40):
        ts_val = base_ts + (i * 900)  # 15분 간격
        messages.append({
            "type": "message",
            "user": f"U{10001+i}",
            "text": work_texts[i],
            "ts": f"{ts_val}.000{i:03d}",
            "chat_with": chat_channels[i % len(chat_channels)]
        })

    # --- 잡담 메시지 10개 생성 ---
    for j in range(10):
        ts_val = base_ts + (40 * 900) + (j * 600)  # 업무 40개 이후, 10분 간격
        messages.append({
            "type": "message",
            "user": f"U20001{j}",
            "text": casual_texts[j],
            "ts": f"{ts_val}.100{j:02d}",
            "chat_with": "잡담 채널"
        })

    return messages


############################################
# --- Dummy WebClient ---
############################################
class DummyWebClient:
    def __init__(self, token=None):
        self.token = token

    def conversations_history(self, channel, limit=50):
        all_messages = generate_dummy_messages()
        return {"messages": all_messages[:limit]}


############################################
# --- 필터링 함수 ---
############################################
def filter_report_messages(messages, keywords):
    """주간보고서 키워드가 포함된 메시지만 필터링"""
    filtered = [msg for msg in messages if any(keyword in msg.get("text", "") for keyword in keywords)]
    removed = [msg for msg in messages if msg not in filtered]
    return filtered, removed


############################################
# --- 월 기준 주차 계산 함수 ---
############################################
def get_month_week(dt: datetime) -> int:
    """해당 날짜가 그 달의 몇 번째 주인지 계산"""
    first_day = dt.replace(day=1)
    first_weekday = first_day.weekday()  # 월=0, 일=6
    day_offset = dt.day + first_weekday - 1
    return (day_offset // 7) + 1


############################################
# --- Slack 메시지 로드 함수 ---
############################################
def load_slack_messages(SLACK_TOKEN, SLACK_CHANNEL_ID, limit=50):
    docs = []
    removed_docs = []
    try:
        if not SLACK_TOKEN or not SLACK_CHANNEL_ID:
            print("⚠️ Slack 설정 없음, 건너뜁니다.")
        else:
            print("🔄 Slack에서 메시지 가져오기...")
            slack_client = DummyWebClient(token=SLACK_TOKEN)
            resp = slack_client.conversations_history(channel=SLACK_CHANNEL_ID, limit=limit)

            # 1. 메시지 가져오기
            all_messages = resp.get("messages", [])

            # 2. 날짜 필터 (2025-09-15 ~ 2025-09-18)
            start_date = datetime(2025, 9, 15)
            end_date = datetime(2025, 9, 18, 23, 59, 59)

            ranged_messages = []
            for msg in all_messages:
                ts_str = msg.get("ts", "0").split(".")[0]
                dt = datetime.fromtimestamp(int(ts_str))
                if start_date <= dt <= end_date:
                    ranged_messages.append(msg)

            # 3. 키워드 필터링 적용
            filtered_messages, removed_messages = filter_report_messages(ranged_messages, REPORT_KEYWORDS)

            # 4. Document 변환 (필터링 통과)
            for msg in filtered_messages:
                docs.append(_convert_to_document(msg))

            # 5. Document 변환 (필터링 제외)
            for msg in removed_messages:
                removed_docs.append(_convert_to_document(msg))

            print(f"✅ Slack {len(docs)}개 로드 완료. (필터링+날짜 적용됨)")
            print(f"⚠️ {len(removed_docs)}개 메시지는 필터링에서 제외됨.")
    except Exception as e:
        print(f"❌ Slack 오류: {e}")
    return docs, removed_docs


############################################
# --- Helper: 메시지를 Document로 변환 ---
############################################
def _convert_to_document(msg):
    text = msg.get("text", "")
    ts_str = msg.get("ts", "0").split(".")[0]
    ts_int = int(ts_str)
    dt = datetime.fromtimestamp(ts_int)
    date_str = dt.strftime("%Y-%m-%d")
    time_str = dt.strftime("%H:%M:%S")

    # real_date는 ts 기반 날짜
    real_date = date_str

    # 월 기준 주차 계산
    month_week = get_month_week(dt)
    month_abbr = dt.strftime("%b").lower()
    week_id = f"{dt.year}-{month_abbr}-{month_week}"

    return Document(
        page_content=text,
        metadata={
            "source": "slack",
            "user": msg.get("user", ""),
            "chat_with": msg.get("chat_with", "unknown"),
            #"date": date_str,
            "time": time_str,
            "date": real_date,
            "week-id": week_id
        },
    )


############################################
# --- 실행 테스트 ---
############################################
if __name__ == "__main__":
    docs, removed_docs = load_slack_messages(SLACK_TOKEN, SLACK_CHANNEL_ID, limit=50)

    print("\n📌 필터링된 메시지 (docs):")
    for d in docs[:5]:
        print(d)

    print("\n❌ 제외된 메시지 (removed_docs):")
    for d in removed_docs[:5]:
        print(d)



🔄 Slack에서 메시지 가져오기...
✅ Slack 40개 로드 완료. (필터링+날짜 적용됨)
⚠️ 10개 메시지는 필터링에서 제외됨.

📌 필터링된 메시지 (docs):
page_content='오늘 회의는 오후 3시에 시작합니다.' metadata={'source': 'slack', 'user': 'U10001', 'chat_with': '개발팀 채널', 'time': '09:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}
page_content='회의 자료는 구글 드라이브에 올려놨어요.' metadata={'source': 'slack', 'user': 'U10002', 'chat_with': 'QA팀 채널', 'time': '09:15:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}
page_content='프로젝트 일정표 업데이트했습니다.' metadata={'source': 'slack', 'user': 'U10003', 'chat_with': '운영팀 채널', 'time': '09:30:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}
page_content='고객사 피드백을 공유드립니다.' metadata={'source': 'slack', 'user': 'U10004', 'chat_with': '기획팀 채널', 'time': '09:45:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}
page_content='내일 오전에 코드 리뷰 진행하겠습니다.' metadata={'source': 'slack', 'user': 'U10005', 'chat_with': '경영지원팀', 'time': '10:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}

❌ 제외된 메시지 (removed_docs):
page_content='오

In [2]:
docs

[Document(metadata={'source': 'slack', 'user': 'U10001', 'chat_with': '개발팀 채널', 'time': '09:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='오늘 회의는 오후 3시에 시작합니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10002', 'chat_with': 'QA팀 채널', 'time': '09:15:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='회의 자료는 구글 드라이브에 올려놨어요.'),
 Document(metadata={'source': 'slack', 'user': 'U10003', 'chat_with': '운영팀 채널', 'time': '09:30:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='프로젝트 일정표 업데이트했습니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10004', 'chat_with': '기획팀 채널', 'time': '09:45:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='고객사 피드백을 공유드립니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10005', 'chat_with': '경영지원팀', 'time': '10:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='내일 오전에 코드 리뷰 진행하겠습니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10006', 'chat_with': '프로젝

In [3]:
import os
from dotenv import load_dotenv
from notion_client import Client
from langchain.docstore.document import Document
from typing import List
from datetime import datetime

############################################
# --- .env 로드 ---
############################################
load_dotenv()
NOTION_TOKEN = os.getenv("NOTION_TOKEN", "")

if not NOTION_TOKEN:
    raise RuntimeError("NOTION_TOKEN이 필요합니다.")

nclient = Client(auth=NOTION_TOKEN)

############################################
# --- 유틸 함수 ---
############################################
def _pt(rt_list):
    """Rich text 리스트에서 plain_text만 추출"""
    return "".join([t.get("plain_text", "") for t in (rt_list or [])])

def _flatten_block(block):
    """Notion block → Markdown 변환"""
    t = block["type"]
    b = block[t]
    if t == "paragraph":
        return _pt(b.get("rich_text"))
    if t.endswith("_heading"):
        return "# " + _pt(b.get("rich_text"))
    if t in ("bulleted_list_item", "numbered_list_item", "to_do"):
        return "- " + _pt(b.get("rich_text"))
    if t == "quote":
        return "> " + _pt(b.get("rich_text"))
    if t == "code":
        txt = b.get("rich_text", [{}])[0].get("plain_text", "")
        lang = b.get("language", "")
        return f"```{lang}\n{txt}\n```"
    if t == "callout":
        return "💡 " + _pt(b.get("rich_text"))
    if t == "toggle":
        return _pt(b.get("rich_text"))  # children은 별도 처리
    if t == "equation":
        return "$" + b.get("expression", "") + "$"
    if t == "table_row":
        cells = [_pt(cell) for cell in b.get("cells", [])]
        return " | ".join(cells)
    return ""

def _walk_children(block_id, acc: List[str]):
    """자식 블록들을 재귀적으로 순회하며 Markdown 텍스트화"""
    children = nclient.blocks.children.list(block_id=block_id)
    while True:
        for b in children["results"]:
            acc.append(_flatten_block(b))
            if b.get("has_children"):
                _walk_children(b["id"], acc)
        if not children.get("has_more"):
            break
        children = nclient.blocks.children.list(
            block_id=block_id,
            start_cursor=children["next_cursor"]
        )

def notion_page_to_markdown(page_id: str) -> str:
    """단일 Notion 페이지를 Markdown 문자열로 변환"""
    out = []
    _walk_children(page_id, out)
    md = "\n".join(filter(None, out)).strip()
    return md

############################################
# --- 월 기준 주차 계산 함수 ---
############################################
def get_month_week(dt: datetime) -> int:
    """해당 날짜가 그 달의 몇 번째 주인지 계산"""
    first_day = dt.replace(day=1)
    first_weekday = first_day.weekday()   # 월=0, 일=6
    day_offset = dt.day + first_weekday - 1
    return (day_offset // 7) + 1

############################################
# --- Document 변환 함수 ---
############################################
def fetch_single_page(page_id: str, 작성자: str = "윤소현") -> Document:
    """Notion 페이지 하나를 Document 형식으로 변환"""
    md = notion_page_to_markdown(page_id)

    # 오늘 날짜와 월 기준 주차 정보 추가
    today = datetime.today()
    real_date = today.strftime("%Y-%m-%d")

    month_week = get_month_week(today)
    month_abbr = today.strftime("%b").lower()  # 예: "Sep" -> "sep"
    week_id = f"{today.year}-{month_abbr}-{month_week}"

    return Document(
        page_content=md,
        metadata={
            "source": "notion",     # ✅ notion_dummy → notion
            "작성자": 작성자,
            "date": real_date,
            "week-id": week_id,
        },
    )

############################################
# --- 여러 페이지 로딩 ---
############################################
def load_notion_pages():
    docs = []
    try:
        print("🔄 Notion 회의록 로드 중...")
        PAGE_IDS = [
            "271766ec530e807ead4bf7456c36ad7f", # 회의록1
            "271766ec530e80deac99d059629ebcbc", # 회의록2
        ] 

        docs: List[Document] = []
        for pid in PAGE_IDS:
            docs.append(fetch_single_page(pid))
        print(f"✅ Notion {len(docs)}개 로드 완료.")
    except Exception as e:
        print(f"❌ Notion 오류: {e}")
    return docs

############################################
# --- 실행 테스트 ---
############################################
if __name__ == "__main__":
    docs = load_notion_pages()
    for d in docs:
        print(d)


🔄 Notion 회의록 로드 중...
✅ Notion 2개 로드 완료.
page_content='제목: 2025년 9월 정기 전략 기획 회의
일시: 2025년 9월 5일 14:00 ~ 17:00
장소: 본사 3층 대회의실
참석자: 대표이사, 전략기획팀장, 개발팀장, 영업팀장, 마케팅팀장, 인사팀장, 재무팀장, 프로젝트 매니저 외 10명
이번 회의는 2025년 하반기 회사의 핵심 전략 과제를 점검하고, 진행 중인 프로젝트들의 리스크와 성과 지표를 확인하며, 향후 6개월간의 실행 계획을 구체화하기 위해 열렸다. 특히 최근 급격히 변화하는 시장 환경과 경쟁사 동향을 반영하여 기존 전략을 보완하고, 각 부서 간 협업 체계를 강화하는 방안이 주요 논의 포인트였다.
- 2025년 상반기 실적 리뷰 및 문제점 분석
- 신규 AI 기반 서비스 출시 일정 및 마케팅 전략 검토
- 해외 진출 전략 및 파트너십 구축 현황
- 인력 충원 및 조직 문화 개선 방안
- 비용 절감 및 효율화 방안
가. 상반기 실적 리뷰
- 영업팀장이 발표한 자료에 따르면 매출은 목표 대비 92% 달성에 그쳤으며, 특히 신규 고객 확보율이 예상보다 15% 낮았다.
- 원인은 경쟁사 A사의 공격적인 가격 정책과, 신제품 출시 지연으로 인한 시장 점유율 하락으로 분석되었다.
- 이에 따라 개발팀과 마케팅팀의 협업을 강화하여 제품 출시 시점과 홍보 시점을 일치시키는 조정이 필요하다는 의견이 제시되었다.
나. 신규 서비스 출시 일정
- 개발팀장은 현재 AI 기반 검색 시스템의 베타 테스트가 안정적으로 진행 중이며, 10월 말 공식 출시를 목표로 하고 있다고 보고했다.
- 마케팅팀장은 사전 예약 이벤트와 인플루언서 마케팅을 병행할 계획이라고 밝혔다.
- 대표이사는 출시 후 첫 달의 성과가 매우 중요하므로, 안정성 확보와 동시에 공격적인 프로모션을 진행해야 한다고 강조했다.
다. 해외 진출 전략
- 전략기획팀은 일본과 동남아 시장을 우선 타겟으로 삼고, 현지 파트너사와의 협업 방안을 추진 중이라고 설명했다.
- 특

In [4]:
docs

[Document(metadata={'source': 'notion', '작성자': '윤소현', 'date': '2025-09-18', 'week-id': '2025-sep-3'}, page_content='제목: 2025년 9월 정기 전략 기획 회의\n일시: 2025년 9월 5일 14:00 ~ 17:00\n장소: 본사 3층 대회의실\n참석자: 대표이사, 전략기획팀장, 개발팀장, 영업팀장, 마케팅팀장, 인사팀장, 재무팀장, 프로젝트 매니저 외 10명\n이번 회의는 2025년 하반기 회사의 핵심 전략 과제를 점검하고, 진행 중인 프로젝트들의 리스크와 성과 지표를 확인하며, 향후 6개월간의 실행 계획을 구체화하기 위해 열렸다. 특히 최근 급격히 변화하는 시장 환경과 경쟁사 동향을 반영하여 기존 전략을 보완하고, 각 부서 간 협업 체계를 강화하는 방안이 주요 논의 포인트였다.\n- 2025년 상반기 실적 리뷰 및 문제점 분석\n- 신규 AI 기반 서비스 출시 일정 및 마케팅 전략 검토\n- 해외 진출 전략 및 파트너십 구축 현황\n- 인력 충원 및 조직 문화 개선 방안\n- 비용 절감 및 효율화 방안\n가. 상반기 실적 리뷰\n- 영업팀장이 발표한 자료에 따르면 매출은 목표 대비 92% 달성에 그쳤으며, 특히 신규 고객 확보율이 예상보다 15% 낮았다.\n- 원인은 경쟁사 A사의 공격적인 가격 정책과, 신제품 출시 지연으로 인한 시장 점유율 하락으로 분석되었다.\n- 이에 따라 개발팀과 마케팅팀의 협업을 강화하여 제품 출시 시점과 홍보 시점을 일치시키는 조정이 필요하다는 의견이 제시되었다.\n나. 신규 서비스 출시 일정\n- 개발팀장은 현재 AI 기반 검색 시스템의 베타 테스트가 안정적으로 진행 중이며, 10월 말 공식 출시를 목표로 하고 있다고 보고했다.\n- 마케팅팀장은 사전 예약 이벤트와 인플루언서 마케팅을 병행할 계획이라고 밝혔다.\n- 대표이사는 출시 후 첫 달의 성과가 매우 중요하므로, 안정성 확보와 동시에 공격적인 프로모션을 진행해야 한다고 강조했다.\

In [5]:
import os
import random
from datetime import datetime, timedelta
from langchain.docstore.document import Document


# --- 더미 Item 클래스 (O365에서 반환하는 파일/폴더 흉내) ---
class DummyItem:
    def __init__(self, name, is_file=True, is_folder=False, created=None, modified=None):
        self.name = name
        self.is_file = is_file
        self.is_folder = is_folder

        # created/modified가 없으면 기본값 지정
        self.created = created
        self.modified = modified


# --- 더미 Drive 클래스 ---
class DummyDrive:
    def get_items(self, limit=5):
        # limit 개수만큼 더미 파일 생성
        all_items = [
            DummyItem("메모.txt"),
            DummyItem("회의록.docx"),
            DummyItem("연구보고서.pdf"),
            DummyItem("발표자료.pptx"),
            DummyItem("실험데이터.xlsx"),
        ]
        return all_items[:limit]


# --- 더미 Storage 클래스 ---
class DummyStorage:
    def get_default_drive(self):
        return DummyDrive()


# --- 더미 Account 클래스 (O365.Account 대체) ---
class DummyAccount:
    def __init__(self, credentials, tenant_id=None):
        self.credentials = credentials
        self.tenant_id = tenant_id
        self._authenticated = True  # 항상 인증된 상태라고 가정

    def is_authenticated(self):
        return self._authenticated

    def authenticate(self, scopes=None):
        print("🔑 OneDrive 인증 완료.")
        self._authenticated = True
        return True

    def storage(self):
        return DummyStorage()


############################################
# --- 월 기준 주차 계산 함수 ---
############################################
def get_month_week(dt: datetime) -> int:
    """해당 날짜가 그 달의 몇 번째 주인지 계산"""
    first_day = dt.replace(day=1)
    first_weekday = first_day.weekday()  # 월=0, 일=6
    day_offset = dt.day + first_weekday - 1
    return (day_offset // 7) + 1


############################################
# --- OneDrive 파일 로드 함수 (더미) ---
############################################
def load_onedrive_files(limit=5):
    docs = []
    try:
        print("🔄 OneDrive에서 파일 목록 가져오기...")

        # O365 계정 연결 (더미)
        credentials = ("dummy_client_id", "dummy_client_secret")
        account = DummyAccount(credentials, tenant_id="dummy_tenant")

        if not account.is_authenticated():
            account.authenticate(scopes=["basic", "onedrive_all"])

        # OneDrive 접근 (더미)
        storage = account.storage()
        my_drive = storage.get_default_drive()

        # 루트 폴더 파일/폴더 목록 가져오기
        items = my_drive.get_items(limit=limit)

        # 날짜 범위: 2025-09-15 ~ 2025-09-19
        start_date = datetime(2025, 9, 15)
        end_date = datetime(2025, 9, 19, 23, 59, 59)

        for item in items:
            # created/modified가 없는 경우 랜덤 날짜 생성
            if not item.created:
                random_days = random.randint(0, (end_date - start_date).days)
                item.created = start_date + timedelta(days=random_days)
            if not item.modified:
                item.modified = item.created + timedelta(hours=random.randint(1, 48))

            # real_date = modified 기준 날짜
            real_date = item.modified.strftime("%Y-%m-%d")

            # 월 기준 주차 계산
            month_week = get_month_week(item.modified)
            month_abbr = item.modified.strftime("%b").lower()
            week_id = f"{item.modified.year}-{month_abbr}-{month_week}"

            docs.append(
                Document(
                    page_content=item.name,
                    metadata={
                        "source": "onedrive",
                        "is_file": item.is_file,
                        "is_folder": item.is_folder,
                        "created": str(item.created) if item.created else None,
                        "modified": str(item.modified) if item.modified else None,
                        "date": real_date,
                        "week-id": week_id,
                    },
                )
            )

        print(f"✅ OneDrive {len(docs)}개 로드 완료.")

    except Exception as e:
        print(f"❌ OneDrive 오류: {e}")
    return docs


# --- 실행 테스트 ---
if __name__ == "__main__":
    docs = load_onedrive_files(limit=5)
    for d in docs:
        print(d)



🔄 OneDrive에서 파일 목록 가져오기...
✅ OneDrive 5개 로드 완료.
page_content='메모.txt' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-15 00:00:00', 'modified': '2025-09-15 23:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}
page_content='회의록.docx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-15 00:00:00', 'modified': '2025-09-15 07:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}
page_content='연구보고서.pdf' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00', 'modified': '2025-09-17 08:00:00', 'date': '2025-09-17', 'week-id': '2025-sep-3'}
page_content='발표자료.pptx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-18 00:00:00', 'modified': '2025-09-18 03:00:00', 'date': '2025-09-18', 'week-id': '2025-sep-3'}
page_content='실험데이터.xlsx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-1

In [6]:
docs

[Document(metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-15 00:00:00', 'modified': '2025-09-15 23:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='메모.txt'),
 Document(metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-15 00:00:00', 'modified': '2025-09-15 07:00:00', 'date': '2025-09-15', 'week-id': '2025-sep-3'}, page_content='회의록.docx'),
 Document(metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00', 'modified': '2025-09-17 08:00:00', 'date': '2025-09-17', 'week-id': '2025-sep-3'}, page_content='연구보고서.pdf'),
 Document(metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-18 00:00:00', 'modified': '2025-09-18 03:00:00', 'date': '2025-09-18', 'week-id': '2025-sep-3'}, page_content='발표자료.pptx'),
 Document(metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00',

In [7]:
import os
import random
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain.docstore.document import Document

############################################
# --- .env 로드 ---
############################################
load_dotenv()
DUMMY_ACCESS_TOKEN = os.getenv("DUMMY_ACCESS_TOKEN", "")


############################################
# --- Dummy Response 클래스 ---
############################################
class DummyResponse:
    """requests.get().json() 과 동일한 인터페이스를 가진 더미 응답"""
    def __init__(self, emails):
        self._emails = emails

    def json(self):
        return {"value": self._emails}


############################################
# --- 더미 Outlook 메일 생성기 ---
############################################
def generate_dummy_outlook_emails():
    base_date = datetime(2025, 9, 15)  # 시작 날짜
    end_date = datetime(2025, 9, 19, 23, 59, 59)  # 종료 날짜

    subjects = [
        "팀 회의 일정 공지",
        "월간 보고서 제출 요청",
        "휴가 신청 승인",
        "시스템 점검 안내",
        "고객사 미팅 일정 확인",
        "비용 정산 안내",
        "보안 교육 이수 안내",
        "신규 프로젝트 착수 보고",
        "연차 사용 안내",
        "회의 자료 공유"
    ]

    bodies = [
        "내일 오후 3시에 회의실 B에서 팀 회의 예정",
        "이번 달 보고서는 금요일까지 제출 바랍니다",
        "홍길동 님의 9월 20일~22일 휴가 신청 승인",
        "토요일 00시~06시 이메일 서버 점검 예정",
        "ABC사 미팅 9월 25일 오전 10시, 본사 회의실 A",
        "출장비 내역 ERP에 업로드 부탁드립니다",
        "정보 보안 교육은 9월 30일까지 완료해야 합니다",
        "프로젝트 'AI 기반 검색 시스템' 10월 1일 시작",
        "10월 첫째 주 연휴 기간 연차 사용 계획 공유 바랍니다",
        "오늘 회의 발표 자료 OneDrive 링크 공유"
    ]

    emails = []
    for subject, body in zip(subjects, bodies):
        # 2025-09-15 ~ 19 사이 랜덤 날짜 생성
        rand_days = random.randint(0, (end_date - base_date).days)
        rand_dt = base_date + timedelta(days=rand_days, hours=random.randint(9, 18))
        emails.append({
            "subject": subject,
            "bodyPreview": body,
            "createdDateTime": rand_dt.isoformat()
        })
    return emails


############################################
# --- 더미 requests.get ---
############################################
def dummy_get(url, headers=None, limit=5):
    print(f"⚠️ 요청 실행: {url} (headers={headers})")
    dummy_emails = generate_dummy_outlook_emails()[:limit]
    return DummyResponse(dummy_emails)


############################################
# --- Outlook 더미 로딩 함수 ---
############################################
def load_outlook_emails(DUMMY_ACCESS_TOKEN, limit=5):
    import requests  # 내부에서 임포트 후 덮어쓰기
    docs = []
    try:
        print("🔄 Outlook에서 메일 가져오기...")

        # 토큰
        token = DUMMY_ACCESS_TOKEN

        # 메일 요청 URL 구성
        url = f"https://graph.microsoft.com/v1.0/me/mailFolders/Inbox/messages?$top={limit}"

        # API 요청 헤더
        headers = {"Authorization": f"Bearer {token}"}

        # requests.get을 더미 함수로 교체
        requests.get = lambda url, headers=None: dummy_get(url, headers=headers, limit=limit)

        # API 요청 (더미 호출)
        resp = requests.get(url, headers=headers).json()

        # 응답 JSON에서 메일 추출
        for mail in resp.get("value", []):
            subject = mail.get("subject", "")
            body_preview = mail.get("bodyPreview", "")

            # 날짜 정보
            created_str = mail.get("createdDateTime")
            if created_str:
                dt = datetime.fromisoformat(created_str)
            else:
                dt = datetime(2025, 9, 15, 9, 0)  # fallback

            real_date = dt.strftime("%Y-%m-%d")

            # 월 기준 주차 계산
            first_day = dt.replace(day=1)
            first_weekday = first_day.weekday()
            day_offset = dt.day + first_weekday - 1
            month_week = (day_offset // 7) + 1
            month_abbr = dt.strftime("%b").lower()
            week_id = f"{dt.year}-{month_abbr}-{month_week}"

            docs.append(
                Document(
                    page_content=body_preview,
                    metadata={
                        "source": f"outlook",
                        #"source": f"outlook:{subject}",
                        "date": real_date,
                        "week-id": week_id,
                        "raw_created": created_str
                    }
                )
            )

        print(f"✅ Outlook {len(docs)}개 로드 완료.")

    except Exception as e:
        print(f"❌ Outlook 오류: {e}")

    return docs


############################################
# 실행 예시
############################################
if __name__ == "__main__":
    docs = load_outlook_emails(DUMMY_ACCESS_TOKEN, limit=10)
    for dArithmeticError in docs:
        print(d)



🔄 Outlook에서 메일 가져오기...
⚠️ 요청 실행: https://graph.microsoft.com/v1.0/me/mailFolders/Inbox/messages?$top=10 (headers={'Authorization': 'Bearer dummy-access-token'})
✅ Outlook 10개 로드 완료.
page_content='실험데이터.xlsx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00', 'modified': '2025-09-18 04:00:00', 'date': '2025-09-18', 'week-id': '2025-sep-3'}
page_content='실험데이터.xlsx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00', 'modified': '2025-09-18 04:00:00', 'date': '2025-09-18', 'week-id': '2025-sep-3'}
page_content='실험데이터.xlsx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00', 'modified': '2025-09-18 04:00:00', 'date': '2025-09-18', 'week-id': '2025-sep-3'}
page_content='실험데이터.xlsx' metadata={'source': 'onedrive', 'is_file': True, 'is_folder': False, 'created': '2025-09-17 00:00:00', 'modified': '2025-09-18 04:00:00', 'date': '2025-09-18',

In [8]:
docs

[Document(metadata={'source': 'outlook', 'date': '2025-09-17', 'week-id': '2025-sep-3', 'raw_created': '2025-09-17T17:00:00'}, page_content='내일 오후 3시에 회의실 B에서 팀 회의 예정'),
 Document(metadata={'source': 'outlook', 'date': '2025-09-18', 'week-id': '2025-sep-3', 'raw_created': '2025-09-18T12:00:00'}, page_content='이번 달 보고서는 금요일까지 제출 바랍니다'),
 Document(metadata={'source': 'outlook', 'date': '2025-09-16', 'week-id': '2025-sep-3', 'raw_created': '2025-09-16T17:00:00'}, page_content='홍길동 님의 9월 20일~22일 휴가 신청 승인'),
 Document(metadata={'source': 'outlook', 'date': '2025-09-16', 'week-id': '2025-sep-3', 'raw_created': '2025-09-16T17:00:00'}, page_content='토요일 00시~06시 이메일 서버 점검 예정'),
 Document(metadata={'source': 'outlook', 'date': '2025-09-16', 'week-id': '2025-sep-3', 'raw_created': '2025-09-16T14:00:00'}, page_content='ABC사 미팅 9월 25일 오전 10시, 본사 회의실 A'),
 Document(metadata={'source': 'outlook', 'date': '2025-09-19', 'week-id': '2025-sep-3', 'raw_created': '2025-09-19T10:00:00'}, page_content='출장비 내

In [9]:
############################################
# --- 전체 문서 로딩 ---
############################################
def load_all_documents():
    summary = {}
    all_docs = []

    # Slack
    slack_docs, removed_slack_docs = load_slack_messages(SLACK_TOKEN, SLACK_CHANNEL_ID, limit=50)
    summary["Slack"] = len(slack_docs)
    for d in slack_docs:
        all_docs.append(
            Document(
                page_content=d.page_content,
                metadata={k: d.metadata.get(k) for k in ["source", "real_date", "week-id"] if k in d.metadata}
            )
        )

    # Notion
    notion_docs = load_notion_pages()
    summary["Notion"] = len(notion_docs)
    for d in notion_docs:
        all_docs.append(
            Document(
                page_content=d.page_content,
                metadata={k: d.metadata.get(k) for k in ["source", "real_date", "week-id"] if k in d.metadata}
            )
        )

    # OneDrive
    onedrive_docs = load_onedrive_files(limit=5)
    summary["OneDrive"] = len(onedrive_docs)
    for d in onedrive_docs:
        all_docs.append(
            Document(
                page_content=d.page_content,
                metadata={k: d.metadata.get(k) for k in ["source", "real_date", "week-id"] if k in d.metadata}
            )
        )

    # Outlook
    outlook_docs = load_outlook_emails(DUMMY_ACCESS_TOKEN, limit=5)
    summary["Outlook"] = len(outlook_docs)
    for d in outlook_docs:
        all_docs.append(
            Document(
                page_content=d.page_content,
                metadata={k: d.metadata.get(k) for k in ["source", "real_date", "week-id"] if k in d.metadata}
            )
        )

    return all_docs, summary


In [10]:
# 실행
if __name__ == "__main__":
    docs, summary = load_all_documents()
    print("\n=== 📊 문서 로딩 결과 요약 ===")
    for source, count in summary.items():
        print(f"{source}: {count}개")
    print(f"총 {len(docs)}개 문서 로드 완료.\n")

    for d in docs[:5]:
        print(f"▶ {d.metadata} | {d.page_content[:50]}...")

🔄 Slack에서 메시지 가져오기...
✅ Slack 40개 로드 완료. (필터링+날짜 적용됨)
⚠️ 10개 메시지는 필터링에서 제외됨.
🔄 Notion 회의록 로드 중...
✅ Notion 2개 로드 완료.
🔄 OneDrive에서 파일 목록 가져오기...
✅ OneDrive 5개 로드 완료.
🔄 Outlook에서 메일 가져오기...
⚠️ 요청 실행: https://graph.microsoft.com/v1.0/me/mailFolders/Inbox/messages?$top=5 (headers={'Authorization': 'Bearer dummy-access-token'})
✅ Outlook 5개 로드 완료.

=== 📊 문서 로딩 결과 요약 ===
Slack: 40개
Notion: 2개
OneDrive: 5개
Outlook: 5개
총 52개 문서 로드 완료.

▶ {'source': 'slack', 'week-id': '2025-sep-3'} | 오늘 회의는 오후 3시에 시작합니다....
▶ {'source': 'slack', 'week-id': '2025-sep-3'} | 회의 자료는 구글 드라이브에 올려놨어요....
▶ {'source': 'slack', 'week-id': '2025-sep-3'} | 프로젝트 일정표 업데이트했습니다....
▶ {'source': 'slack', 'week-id': '2025-sep-3'} | 고객사 피드백을 공유드립니다....
▶ {'source': 'slack', 'week-id': '2025-sep-3'} | 내일 오전에 코드 리뷰 진행하겠습니다....


In [11]:
# --- 필요한 패키지 설치 ---
#!pip install requests langchain

# --- 필요한 import ---
from datetime import datetime
import os
import requests
from langchain.docstore.document import Document
from notion_client import Client
from typing import List, Dict
from langchain.schema import Document
from dotenv import load_dotenv


############################################
# --- Slack (더미) ---
############################################

# --- 더미 토큰/채널 설정 ---
# --- .env 로드 ---
load_dotenv()
# --- 환경변수에서 읽기 ---
SLACK_TOKEN = os.getenv("SLACK_TOKEN", "")
SLACK_CHANNEL_ID = os.getenv("SLACK_CHANNEL_ID", "")


# --- 주간보고서 관련 키워드 (30개) ---
REPORT_KEYWORDS = [
    "회의", "미팅", "일정", "계획", "진행", "완료", "성과", "목표", "보고",
    "결과", "피드백", "공유", "자료", "문서", "슬라이드", "첨부", "업로드",
    "검토", "확인", "협의", "토론", "논의", "배포", "개발", "테스트",
    "이슈", "문제", "해결", "개선", "요청"
]

############################################
# --- 더미 메시지 생성 함수 ---
############################################
def generate_dummy_messages():
    """
    50개의 더미 Slack 메시지를 생성해서 반환
    """
    return [
        # --- 1~10 ---
        {"type": "message", "user": "U10001", "text": "오늘 회의는 오후 3시에 시작합니다.", "ts": "1694958400.000200", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10002", "text": "회의 자료는 구글 드라이브에 올려놨어요.", "ts": "1694958600.000300", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10003", "text": "네 확인했습니다!", "ts": "1694958700.000400", "chat_with": "김철수"},
        {"type": "message", "user": "U10004", "text": "프로젝트 일정표 업데이트했습니다.", "ts": "1694958800.000500", "chat_with": "프로젝트A 그룹"},
        {"type": "message", "user": "U10005", "text": "오늘 점심은 12시에 먹어요.", "ts": "1694958900.000600", "chat_with": "마케팅팀 채널"},
        {"type": "message", "user": "U10006", "text": "고객사 피드백을 공유드립니다.", "ts": "1694959000.000700", "chat_with": "프로젝트A 그룹"},
        {"type": "message", "user": "U10007", "text": "내일 오전에 코드 리뷰 진행하겠습니다.", "ts": "1694959100.000800", "chat_with": "QA팀 채널"},
        {"type": "message", "user": "U10008", "text": "회의실 예약은 제가 해두었습니다.", "ts": "1694959200.000900", "chat_with": "총무팀"},
        {"type": "message", "user": "U10009", "text": "배포는 오후 6시에 시작할 예정입니다.", "ts": "1694959300.001000", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10010", "text": "다들 수고 많으셨습니다!", "ts": "1694959400.001100", "chat_with": "개발팀 채널"},

        # --- 11~20 ---
        {"type": "message", "user": "U10011", "text": "신규 기능 개발 일정이 확정되었습니다.", "ts": "1694959500.001200", "chat_with": "프로젝트B 그룹"},
        {"type": "message", "user": "U10012", "text": "이번 주 목표는 성능 개선입니다.", "ts": "1694959600.001300", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10013", "text": "테스트 환경에서 문제가 발견되었습니다.", "ts": "1694959700.001400", "chat_with": "QA팀 채널"},
        {"type": "message", "user": "U10014", "text": "이슈 해결 방안을 논의해야 합니다.", "ts": "1694959800.001500", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10015", "text": "다음 주 미팅 안건을 정리했습니다.", "ts": "1694959900.001600", "chat_with": "기획팀 채널"},
        {"type": "message", "user": "U10016", "text": "고객 요청사항을 반영한 문서를 첨부합니다.", "ts": "1694960000.001700", "chat_with": "고객지원팀"},
        {"type": "message", "user": "U10017", "text": "이번 릴리즈 결과를 보고드립니다.", "ts": "1694960100.001800", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10018", "text": "팀원별 업무 진행 상황을 공유합니다.", "ts": "1694960200.001900", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10019", "text": "문제 해결 후 재배포가 필요합니다.", "ts": "1694960300.002000", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10020", "text": "이번 주 성과를 정리한 슬라이드를 업로드했습니다.", "ts": "1694960400.002100", "chat_with": "경영지원팀"},

        # --- 21~30 ---
        {"type": "message", "user": "U10021", "text": "이번 주 회의록을 검토해 주세요.", "ts": "1694960500.002200", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10022", "text": "QA팀에서 테스트 결과 보고서를 공유했습니다.", "ts": "1694960600.002300", "chat_with": "QA팀 채널"},
        {"type": "message", "user": "U10023", "text": "신규 프로젝트 계획안을 업로드했습니다.", "ts": "1694960700.002400", "chat_with": "프로젝트C 그룹"},
        {"type": "message", "user": "U10024", "text": "고객사 미팅 피드백을 반영해야 합니다.", "ts": "1694960800.002500", "chat_with": "영업팀 채널"},
        {"type": "message", "user": "U10025", "text": "서버 장애 문제를 해결 완료했습니다.", "ts": "1694960900.002600", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10026", "text": "팀 목표 달성을 위해 개선 작업이 필요합니다.", "ts": "1694961000.002700", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10027", "text": "자료 검토 후 의견을 부탁드립니다.", "ts": "1694961100.002800", "chat_with": "기획팀 채널"},
        {"type": "message", "user": "U10028", "text": "이번 주 요청사항을 정리했습니다.", "ts": "1694961200.002900", "chat_with": "고객지원팀"},
        {"type": "message", "user": "U10029", "text": "배포 완료 후 최종 결과를 보고합니다.", "ts": "1694961300.003000", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10030", "text": "성과 발표 슬라이드를 회의 전에 공유하겠습니다.", "ts": "1694961400.003100", "chat_with": "경영지원팀"},

        # --- 31~40 ---
        {"type": "message", "user": "U10031", "text": "내일 회의 안건을 정리했습니다.", "ts": "1694961500.003200", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10032", "text": "테스트 진행 상황을 보고드립니다.", "ts": "1694961600.003300", "chat_with": "QA팀 채널"},
        {"type": "message", "user": "U10033", "text": "신규 기능 개발 완료 보고서를 첨부합니다.", "ts": "1694961700.003400", "chat_with": "프로젝트D 그룹"},
        {"type": "message", "user": "U10034", "text": "회의 피드백을 반영한 수정안을 공유합니다.", "ts": "1694961800.003500", "chat_with": "기획팀 채널"},
        {"type": "message", "user": "U10035", "text": "이번 주 일정이 변경되었습니다.", "ts": "1694961900.003600", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10036", "text": "고객 요청에 따른 추가 개발이 필요합니다.", "ts": "1694962000.003700", "chat_with": "고객지원팀"},
        {"type": "message", "user": "U10037", "text": "프로젝트 리스크 검토 회의를 잡겠습니다.", "ts": "1694962100.003800", "chat_with": "프로젝트E 그룹"},
        {"type": "message", "user": "U10038", "text": "성과 발표 준비 자료를 업로드했습니다.", "ts": "1694962200.003900", "chat_with": "경영지원팀"},
        {"type": "message", "user": "U10039", "text": "이번 주 배포 계획을 정리했습니다.", "ts": "1694962300.004000", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10040", "text": "회의록을 기반으로 개선 사항을 정리했습니다.", "ts": "1694962400.004100", "chat_with": "개발팀 채널"},

        # --- 41~50 ---
        {"type": "message", "user": "U10041", "text": "테스트 중 발견된 문제를 보고합니다.", "ts": "1694962500.004200", "chat_with": "QA팀 채널"},
        {"type": "message", "user": "U10042", "text": "문서 검토 후 서명을 부탁드립니다.", "ts": "1694962600.004300", "chat_with": "기획팀 채널"},
        {"type": "message", "user": "U10043", "text": "성과 달성을 위한 팀 목표를 공유합니다.", "ts": "1694962700.004400", "chat_with": "경영지원팀"},
        {"type": "message", "user": "U10044", "text": "이번 주 완료된 작업 리스트를 보고합니다.", "ts": "1694962800.004500", "chat_with": "운영팀 채널"},
        {"type": "message", "user": "U10045", "text": "회의에서 나온 요청사항을 정리했습니다.", "ts": "1694962900.004600", "chat_with": "프로젝트F 그룹"},
        {"type": "message", "user": "U10046", "text": "신규 프로젝트 킥오프 미팅 일정을 공유합니다.", "ts": "1694963000.004700", "chat_with": "기획팀 채널"},
        {"type": "message", "user": "U10047", "text": "개발팀에서 개선 방안을 마련했습니다.", "ts": "1694963100.004800", "chat_with": "개발팀 채널"},
        {"type": "message", "user": "U10048", "text": "테스트 완료 후 결과 보고서를 제출합니다.", "ts": "1694963200.004900", "chat_with": "QA팀 채널"},
        {"type": "message", "user": "U10049", "text": "회의 자료와 슬라이드를 첨부합니다.", "ts": "1694963300.005000", "chat_with": "프로젝트G 그룹"},
        {"type": "message", "user": "U10050", "text": "이번 주 프로젝트 진행 상황을 요약했습니다.", "ts": "1694963400.005100", "chat_with": "운영팀 채널"},
    ]



############################################
# --- Dummy WebClient ---
############################################
class DummyWebClient:
    def __init__(self, token=None):
        self.token = token

    def conversations_history(self, channel, limit=50):
        all_messages = generate_dummy_messages()
        return {"messages": all_messages[:limit]}


############################################
# --- 필터링 함수 ---
############################################
def filter_report_messages(messages, keywords):
    """주간보고서 키워드가 포함된 메시지만 필터링"""
    filtered = [msg for msg in messages if any(keyword in msg.get("text", "") for keyword in keywords)]
    removed = [msg for msg in messages if msg not in filtered]
    return filtered, removed


############################################
# --- Slack 메시지 로드 함수 ---
############################################
def load_slack_messages(SLACK_TOKEN, SLACK_CHANNEL_ID, limit=50):
    docs = []
    removed_docs = []
    try:
        if not SLACK_TOKEN or not SLACK_CHANNEL_ID:
            print("⚠️ Slack 설정 없음, 건너뜁니다.")
        else:
            print("🔄 (Dummy) Slack에서 메시지 가져오기...")
            slack_client = DummyWebClient(token=SLACK_TOKEN)
            resp = slack_client.conversations_history(channel=SLACK_CHANNEL_ID, limit=limit)

            # 1. 메시지 가져오기
            all_messages = resp.get("messages", [])

            # 2. 필터링 적용
            filtered_messages, removed_messages = filter_report_messages(all_messages, REPORT_KEYWORDS)

            # 3. Document 변환 (필터링 통과)
            for msg in filtered_messages:
                docs.append(_convert_to_document(msg))

            # 4. Document 변환 (필터링 제거된 것)
            for msg in removed_messages:
                removed_docs.append(_convert_to_document(msg))

            print(f"✅ (Dummy) Slack {len(docs)}개 로드 완료. (필터링 적용됨)")
            print(f"⚠️ {len(removed_docs)}개 메시지는 필터링에서 제외됨.")
    except Exception as e:
        print(f"❌ Slack 오류: {e}")
    return docs, removed_docs


############################################
# --- Helper: 메시지를 Document로 변환 ---
############################################
def _convert_to_document(msg):
    text = msg.get("text", "")
    ts_str = msg.get("ts", "0").split(".")[0]
    ts_int = int(ts_str)
    dt = datetime.fromtimestamp(ts_int)
    date_str = dt.strftime("%Y-%m-%d")
    time_str = dt.strftime("%H:%M:%S")

    return Document(
        page_content=text,
        metadata={
            "source": "slack",
            "user": msg.get("user", ""),
            "chat_with": msg.get("chat_with", "unknown"),
            "date": date_str,
            "time": time_str,
        },
    )


############################################
# --- 실행 테스트 ---
############################################
if __name__ == "__main__":
    docs, removed_docs = load_slack_messages(SLACK_TOKEN, SLACK_CHANNEL_ID, limit=50)

    print("\n📌 필터링된 메시지 (docs):")
    #for d in docs:
    #    print(d)

    print("\n❌ 필터링 제외된 메시지 (removed_docs):")
    for d in removed_docs:
        print(d)

🔄 (Dummy) Slack에서 메시지 가져오기...
✅ (Dummy) Slack 48개 로드 완료. (필터링 적용됨)
⚠️ 2개 메시지는 필터링에서 제외됨.

📌 필터링된 메시지 (docs):

❌ 필터링 제외된 메시지 (removed_docs):
page_content='오늘 점심은 12시에 먹어요.' metadata={'source': 'slack', 'user': 'U10005', 'chat_with': '마케팅팀 채널', 'date': '2023-09-17', 'time': '22:55:00'}
page_content='다들 수고 많으셨습니다!' metadata={'source': 'slack', 'user': 'U10010', 'chat_with': '개발팀 채널', 'date': '2023-09-17', 'time': '23:03:20'}


In [12]:
len(docs)

48

In [13]:
docs

[Document(metadata={'source': 'slack', 'user': 'U10001', 'chat_with': '개발팀 채널', 'date': '2023-09-17', 'time': '22:46:40'}, page_content='오늘 회의는 오후 3시에 시작합니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10002', 'chat_with': '개발팀 채널', 'date': '2023-09-17', 'time': '22:50:00'}, page_content='회의 자료는 구글 드라이브에 올려놨어요.'),
 Document(metadata={'source': 'slack', 'user': 'U10003', 'chat_with': '김철수', 'date': '2023-09-17', 'time': '22:51:40'}, page_content='네 확인했습니다!'),
 Document(metadata={'source': 'slack', 'user': 'U10004', 'chat_with': '프로젝트A 그룹', 'date': '2023-09-17', 'time': '22:53:20'}, page_content='프로젝트 일정표 업데이트했습니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10006', 'chat_with': '프로젝트A 그룹', 'date': '2023-09-17', 'time': '22:56:40'}, page_content='고객사 피드백을 공유드립니다.'),
 Document(metadata={'source': 'slack', 'user': 'U10007', 'chat_with': 'QA팀 채널', 'date': '2023-09-17', 'time': '22:58:20'}, page_content='내일 오전에 코드 리뷰 진행하겠습니다.'),
 Document(metadata={'source': 'slack', 'user': 

In [14]:
import json

def save_docs_to_json(docs, filename="documents.json"):
    data = []
    for d in docs:
        data.append({
            "page_content": d.page_content,
            "metadata": d.metadata
        })
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ 저장 완료: {filename}")

save_docs_to_json(docs, "documents.json")

# 읽을때
#import json
#from langchain.docstore.document import Document

#def load_docs_from_json(filename="documents.json"):
#    with open(filename, "r", encoding="utf-8") as f:
#        data = json.load(f)
#    docs = [Document(page_content=item["page_content"], metadata=item["metadata"]) for item in data]
#    return docs

# 실행 예시
#docs = load_docs_from_json("documents.json")

✅ 저장 완료: documents.json
